## 目标：提取一都三县的各个h3 栅格的POI 数目信息

In [34]:
import pandas as pd
import h3 

### Step 1. 数据读取 
- 仅保留 店名，类型码，经度，纬度

In [2]:
import pandas as pd
import os
path_1 = "/yourpath/11"
path_2 = "/yourpath/12"
path_3 = "/yourpath/13"
path_4 = "/yourpath/14"
all_files_1 = [os.path.join(path_1, x) for x in os.listdir(path_1)]
all_files_2 = [os.path.join(path_2, x) for x in os.listdir(path_2)]
all_files_3 = [os.path.join(path_3, x) for x in os.listdir(path_3)]
all_files_4 = [os.path.join(path_4, x) for x in os.listdir(path_4)]
all_files = all_files_1 + all_files_2 + all_files_3 + all_files_4
li = []
for filename in all_files:   # 注意 类别码 是字符串格式
    df = pd.read_csv(filename, index_col=None, header = None,encoding= 'shift-JIS',usecols=[0,10,20,21],dtype={10:'str'})
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
data.columns = ['name','category','lon','lat']

### Step 2. 过滤掉 个人电话用户 

In [5]:
data = data[data['category'].notnull()]

In [6]:
data

,name,category,lon,lat
1,（株）アイビックス 大宮工場,1601000,139.589022,35.894541
2,日本郵便輸送（株）さいたま物流センター,2509000,139.586545,35.890433
3,さいたまメモリアルパーク,3571000,139.584468,35.895304
23,中村金物店,1302000,139.593366,35.893898
25,植水郵便局,3821000,139.594448,35.894767
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,139.228760,35.523315
4303504,（有）フリート,3173000,139.227249,35.520674
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,139.227624,35.522019
4303507,宮ヶ瀬湖遊覧船案内所,2303000,139.227624,35.522019


### Step 3. 去重

In [8]:
data = data.drop_duplicates()
data

,name,category,lon,lat
1,（株）アイビックス 大宮工場,1601000,139.589022,35.894541
2,日本郵便輸送（株）さいたま物流センター,2509000,139.586545,35.890433
3,さいたまメモリアルパーク,3571000,139.584468,35.895304
23,中村金物店,1302000,139.593366,35.893898
25,植水郵便局,3821000,139.594448,35.894767
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,139.228760,35.523315
4303504,（有）フリート,3173000,139.227249,35.520674
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,139.227624,35.522019
4303507,宮ヶ瀬湖遊覧船案内所,2303000,139.227624,35.522019


### Step 4. 坐标偏移矫正，从日本坐标系 矫正到 世界坐标系

In [9]:
def get_global_lat(jlat,jlon):
    glat = (jlat - jlat * 0.00010695 + jlon * 0.000017464 + 0.0046017)
    return glat
def get_global_lon(jlat,jlon):
    glon = jlon - jlat * 0.000046038 - jlon * 0.000083043 + 0.010040
    return glon
def convert_coords(df):
    df['n_lat'] = df.apply(lambda x: get_global_lat(x['lat'],x['lon']), axis=1)
    df['n_lon'] = df.apply(lambda x: get_global_lon(x['lat'],x['lon']), axis=1)
    return df

data = convert_coords(data)

/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
data = data[['name','category','n_lat','n_lon']]
data.columns = ['name','category','lat','lon']
data

,name,category,lat,lon
1,（株）アイビックス 大宮工場,1601000,35.897742,139.585818
2,日本郵便輸送（株）さいたま物流センター,2509000,35.893634,139.583341
3,さいたまメモリアルパーク,3571000,35.898505,139.581264
23,中村金物店,1302000,35.897099,139.590162
25,植水郵便局,3821000,35.897968,139.591243
...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,35.526549,139.225602
4303504,（有）フリート,3173000,35.523908,139.224092
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,35.525253,139.224467
4303507,宮ヶ瀬湖遊覧船案内所,2303000,35.525253,139.224467


### Step 5. 计算每条记录所在的网格信息

In [35]:
data['hid']  = data.apply(lambda x: h3.geo_to_h3(x['lat'],x['lon'],8), axis=1)

In [36]:
data

,name,category,lat,lon,hid
1,（株）アイビックス 大宮工場,1601000,35.897742,139.585818,882f5a2e59fffff
2,日本郵便輸送（株）さいたま物流センター,2509000,35.893634,139.583341,882f5a2e59fffff
3,さいたまメモリアルパーク,3571000,35.898505,139.581264,882f5a2e59fffff
23,中村金物店,1302000,35.897099,139.590162,882f5a20b7fffff
25,植水郵便局,3821000,35.897968,139.591243,882f5a20b7fffff
...,...,...,...,...,...
4303503,宮ヶ瀬水の郷交流館,3818000,35.526549,139.225602,882f5bc5d7fffff
4303504,（有）フリート,3173000,35.523908,139.224092,882f5bc5d3fffff
4303505,（財）宮ヶ瀬ダム周辺振興財団,3902000,35.525253,139.224467,882f5bc5d3fffff
4303507,宮ヶ瀬湖遊覧船案内所,2303000,35.525253,139.224467,882f5bc5d3fffff


### Step 6. 计算每个网格中总POI 的数目

In [37]:
total_poi_count_df = data.groupby(['hid']).size().reset_index(name='counts')
total_poi_count_df

,hid,counts
0,882e2da413fffff,81
1,882e2da417fffff,6
2,882e2da41bfffff,14
3,882e2da481fffff,1
4,882e2da483fffff,2
...,...,...
15034,882f5e2eb1fffff,1
15035,882f5e2eb3fffff,31
15036,882f5e2eb5fffff,2
15037,882f5e2eb7fffff,9


### Step 6. 提取网格中大型店铺/娱乐场所/餐厅 POI 的数目("娱乐和食事关联"&"大型综合店铺")
![jupyter](./categories.png)

#### 6.1 各类别POI 数据提取

In [38]:
def my_filter(type_code):
    if type_code.startswith("31"):
        return True
    if type_code.startswith("34"):
        return True
    return False
data['newcolumn'] = data.apply(lambda x: my_filter(x['category']), axis=1)

In [39]:
filtered_poi = data.loc[data['newcolumn'] == True]
del filtered_poi['newcolumn']
filtered_poi

,name,category,lat,lon,hid
142,Ｐ．Ｓ．Ｘさいたま西区店,3106000,35.885870,139.556161,882f5a2505fffff
294,船渡,3166000,35.883116,139.554523,882f5a2505fffff
431,和風スナック花,3173000,35.888989,139.596207,882f5a20b1fffff
474,池田屋,3132000,35.888167,139.596186,882f5a20b1fffff
655,銀鮨,3149000,35.890896,139.588314,882f5a20b3fffff
...,...,...,...,...,...
4303455,民芸風食事処・恵山亭,3173000,35.524363,139.223677,882f5bc5d3fffff
4303461,山登,3132000,35.524184,139.223744,882f5bc5d3fffff
4303478,びんずる,3167000,35.523366,139.224143,882f5bc5d3fffff
4303491,田舎料理水の郷,3173000,35.526549,139.225602,882f5bc5d7fffff


#### 娱乐
- ['3101000', '映画館']
- ['3102000', 'プラネタリウム']
- ['3103000', '劇場・寄席']
- ['3104000', 'ライブハウス']
- ['3105000', 'ゲームセンター']
- ['3105001', 'セガワールド']
- ['3105002', 'ナムコ']
- ['3106000', 'パチンコ']
- ['3108000', 'マージャンクラブ']
- ['3111000', 'カラオケ']
- ['3111001', 'シダックス']
- ['3111002', 'ＢＩＧＥＣＨＯ']
- ['3112000', 'ディスコ']
- ['3113000', 'ダンスホール']
- ['3178000', 'その他娯楽']
- ['3179000', '風俗店']

In [48]:
entertainment = set(['3101000','3102000','3103000','3104000','3105000',
 '3105001','3105002','3106000','3108000','3111000',
 '3111001','3111002','3112000','3113000','3178000','3179000'])

In [49]:
entertainment_poi = filtered_poi[filtered_poi['category'].isin(entertainment)]
del entertainment_poi['category']
entertainment_poi = entertainment_poi.drop_duplicates()  # 同一 POI 属于多中娱乐子类
entertainment_poi

,name,lat,lon,hid
142,Ｐ．Ｓ．Ｘさいたま西区店,35.885870,139.556161,882f5a2505fffff
2372,シダックス大宮指扇クラブ,35.914762,139.577922,882f5a2e57fffff
3301,パーラー平成大宮店,35.899263,139.586752,882f5a2e59fffff
3993,マルハン清河寺店,35.934060,139.581318,882f5a2e07fffff
6771,パチンコダスカダスヨ,35.893342,139.582231,882f5a2e59fffff
...,...,...,...,...
4298788,ジリオン中津ジュニア店,35.515279,139.336869,882f5bc727fffff
4299672,旭,35.511147,139.340658,882f5bc721fffff
4300109,ニューダイヤ,35.517933,139.333701,882f5bc727fffff
4300130,パチンコメルシー愛川店,35.528035,139.331000,882f5bc093fffff


#### 餐厅

In [50]:
tmp = filtered_poi[filtered_poi['category'].str.startswith("31")]
restaurant_poi = tmp[tmp['category'].isin(entertainment) == False]
del restaurant_poi['category']
restaurant_poi = restaurant_poi.drop_duplicates()
restaurant_poi

,name,lat,lon,hid
294,船渡,35.883116,139.554523,882f5a2505fffff
431,和風スナック花,35.888989,139.596207,882f5a20b1fffff
474,池田屋,35.888167,139.596186,882f5a20b1fffff
655,銀鮨,35.890896,139.588314,882f5a20b3fffff
656,味の銀鮨,35.890896,139.588314,882f5a20b3fffff
...,...,...,...,...
4303455,民芸風食事処・恵山亭,35.524363,139.223677,882f5bc5d3fffff
4303461,山登,35.524184,139.223744,882f5bc5d3fffff
4303478,びんずる,35.523366,139.224143,882f5bc5d3fffff
4303491,田舎料理水の郷,35.526549,139.225602,882f5bc5d7fffff


#### 大型商店

In [52]:
shopping_mall = filtered_poi[filtered_poi['category'].str.startswith("34")]
del shopping_mall['category']
shopping_mall_poi = shopping_mall.drop_duplicates()
shopping_mall_poi

,name,lat,lon,hid
1226,リサイクルショップ宝島 指扇店,35.912360,139.575497,882f5a2e57fffff
2616,髪飾りの手仕事屋,35.913840,139.572356,882f5a2e57fffff
2642,（株）ＬＪＰコーポレーション,35.916870,139.573022,882f5a2e1dfffff
2775,ライフ 指扇店,35.919943,139.566240,882f5a2e11fffff
2810,１００円ショップキャンドゥライフ指扇店,35.919691,139.565933,882f5a2e11fffff
...,...,...,...,...
4300502,マルエツ中津店,35.514887,139.338253,882f5bc725fffff
4300536,（有）フラワーランド 愛川店,35.520372,139.336326,882f5bc09bfffff
4301057,（株）たからや 半原店,35.537713,139.270207,882f5bc513fffff
4301058,スーパーたからや半原店,35.537713,139.270207,882f5bc513fffff


### Step 7. 计算各个网格中，各种POI 的数目

In [53]:
entertainment_count = entertainment_poi.groupby(['hid']).size().reset_index(name='entertainment_count')
entertainment_count

,hid,entertainment_count
0,882e2da489fffff,1
1,882e2da48bfffff,6
2,882e2da499fffff,1
3,882e2da49dfffff,1
4,882e748093fffff,1
...,...,...
2523,882f5e2dbdfffff,1
2524,882f5e2dd1fffff,1
2525,882f5e2dd3fffff,1
2526,882f5e2eb3fffff,1


In [54]:
restaurant_count = restaurant_poi.groupby(['hid']).size().reset_index(name='restaurant_count')
restaurant_count

,hid,restaurant_count
0,882e2da413fffff,4
1,882e2da41bfffff,1
2,882e2da489fffff,151
3,882e2da48bfffff,91
4,882e2da48dfffff,4
...,...,...
9134,882f5e2dd7fffff,14
9135,882f5e2ddbfffff,9
9136,882f5e2dddfffff,1
9137,882f5e2eb3fffff,2


In [56]:
shopping_mall_count = shopping_mall_poi.groupby(['hid']).size().reset_index(name='shopping_mall_count')
shopping_mall_count

,hid,shopping_mall_count
0,882e2da489fffff,5
1,882e2da48bfffff,9
2,882e2da491fffff,1
3,882e2da497fffff,1
4,882e2da4c5fffff,1
...,...,...
4879,882f5e2dbdfffff,2
4880,882f5e2dd1fffff,2
4881,882f5e2dd5fffff,1
4882,882f5e2dd7fffff,4


### Step 8. 合并导出结果

In [59]:
hid_poi_info = pd.merge(restaurant_count,entertainment_count, how='left', on=['hid'])
hid_poi_info = pd.merge(hid_poi_info, shopping_mall_count, how='left', on=['hid'])
hid_poi_info = pd.merge(total_poi_count_df, hid_poi_info, how='left', on=['hid'])
hid_poi_info = hid_poi_info.fillna(0)
hid_poi_info

,hid,counts,restaurant_count,entertainment_count,shopping_mall_count
0,882e2da413fffff,81,4.0,0.0,0.0
1,882e2da417fffff,6,0.0,0.0,0.0
2,882e2da41bfffff,14,1.0,0.0,0.0
3,882e2da481fffff,1,0.0,0.0,0.0
4,882e2da483fffff,2,0.0,0.0,0.0
...,...,...,...,...,...
15034,882f5e2eb1fffff,1,0.0,0.0,0.0
15035,882f5e2eb3fffff,31,2.0,1.0,0.0
15036,882f5e2eb5fffff,2,0.0,0.0,0.0
15037,882f5e2eb7fffff,9,0.0,0.0,0.0


In [62]:
hid_poi_info.to_csv("./hid_poi_info.csv", encoding='utf-8', index=False)

### Result Visualization 
- **(a)Total POI count distribution**
- **(b)Restaurant_count distribution**
- **(c)Shopping mall count distribution**
- **(d)Entertainment_count distribution**
![jupyter](./poi_vis.png)
